In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
68,ABW,Aruba,2020-05-30,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
210,AFG,Afghanistan,2020-05-30,13659,623,246,11,350.875,16.004,6.319,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
280,AGO,Angola,2020-05-30,77,4,4,0,2.343,0.122,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-29,France,149851.0,0.0,28735.0,0.0,0.0,0.0
0,2020-05-29,China,84168.0,0.0,4654.0,0.0,0.0,0.0
0,2020-05-29,Italy,232416.0,0.0,33274.0,0.0,0.0,0.0
0,2020-05-29,Spain,238442.0,0.0,27139.0,0.0,0.0,0.0
0,2020-05-29,United States,1742733.0,0.0,103211.0,0.0,0.0,0.0
0,2020-05-29,World,5890817.0,0.0,365548.0,0.0,0.0,0.0
0,2020-05-29,United Kingdom,271514.0,0.0,38271.0,0.0,0.0,0.0
0,2020-05-29,Germany,180889.0,0.0,8506.0,0.0,0.0,0.0
0,2020-05-29,Iran,146022.0,0.0,7690.0,0.0,0.0,0.0
0,2020-05-29,Turkey,161024.0,0.0,4460.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-30,France,149851.0,149668.0,28735.0,28714.0,183.0,21.0
0,2020-05-30,China,84168.0,84123.0,4654.0,4638.0,45.0,16.0
0,2020-05-30,Italy,232416.0,232248.0,33274.0,33229.0,168.0,45.0
0,2020-05-29,Spain,238442.0,238564.0,27139.0,27121.0,-122.0,18.0
0,2020-05-30,United States,1742733.0,1747087.0,103211.0,102836.0,-4354.0,375.0
0,2020-05-30,World,5890817.0,5899866.0,365548.0,364891.0,-9049.0,657.0
0,2020-05-30,United Kingdom,271514.0,271222.0,38271.0,38161.0,292.0,110.0
0,2020-05-30,Germany,180889.0,181196.0,8506.0,8489.0,-307.0,17.0
0,2020-05-30,Iran,146022.0,146668.0,7690.0,7677.0,-646.0,13.0
0,2020-05-30,Turkey,161024.0,162120.0,4460.0,4489.0,-1096.0,-29.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")